# Bulk HTTP Posts 

Because Postman can't export all your requests and responses from a bulk run. Python, once again, is superior.

## 1. Upload CSV Data for Record-based Requests

Run the cell below to display the file upload widget (as well as import necessary packages and generate logger to store request/response data). Then click on the widget to upload the .csv file

In [32]:
import pyodbc
import pandas as pd
import requests
import json
import datetime
import re
import io
import os 
import ipywidgets as widgets

logs = pd.DataFrame(columns=["RequestUrl", "RequestBody", 'ResponseCode', "ResponseBody"])

uploader = widgets.FileUpload(accept='*.csv',multiple=False)
display(uploader)

FileUpload(value={}, accept='*.csv', description='Upload')

## 2. Process and Preview CSV Data

Run the cell below to process the selected file onto this notebook.
<b>Note:</b> `ipywidgets` appears to not work currently. For now, read the csv by specifying the file location on your local device until fixed.

In [33]:
#input_file = list(uploader.value.values())[0]
#content = input_file['content']
#content = io.StringIO(content.decode('utf-8'))
#df = pd.read_csv(content)
#df.columns = df.columns.str.lower()
#df = df.fillna('')
#df = df.applymap(str)
#filename = next(iter(uploader.value))

# SPECIFY FILE PATH WHEN IPYWIDGETS ARE DOWN
df = pd.read_csv('Downloads/Lifestone - Velocify Migration WORKING.csv')
df = df.head(20)
df.columns = df.columns.str.lower()
df = df.fillna('')
df = df.applymap(str)
print(df)

   campaignid       uid first name       last name                      email  \
0       10047  VF-00001    Bernard           McKay          bmckay1@wi.rr.com   
1       10047  VF-00002      Brian          Pigney                              
2       10047  VF-00003      Kelsi         Konopka    kelsi.konopka@yahoo.com   
3       10047  VF-00004      shana          thomas         shanamoh@gmail.com   
4       10047  VF-00005      Vijay  Ravidindrababu      Pranaov.r@outlook.com   
5       10047  VF-00006  Angelique          Belder  angiebelder21@hotmail.com   
6       10047  VF-00007    America        QUINTANA     quintana1427@gmail.com   
7       10047  VF-00008      Trent            Hill        bobbyf15e@yahoo.com   
8       10047  VF-00009       Tony          Coomer  tonycoomer@rocketmail.com   
9       10047  VF-00010     Julius        Williams                              
10      10047  VF-00011       Tony        Schaefer      cardbaron@hotmail.com   
11      10047  VF-00012     

C:\Users\Richard\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,21,27,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## 3. Reference to Mapping Variables

I replicated a similar HTTP bulk action to the runner on Postman because of its simplicity, user-friendliness, and dynamic use. Just like Postman, you can create variables that point to the CSV column headers-- substituting them out for the values once the code iterates through each row.

Run the cell below to obtain this notebook's variables (<b>*py_var*</b>). For more clarity, the typical Postman handlebar variables are displayed against the variables used here (under <b>*postman_var*</b>

In [34]:
var = pd.DataFrame(columns = ['headers','postman_var','py_var'])
var['headers'] = df.columns
var['postman_var']= "{{"+df.columns+"}}"
var['py_var']= "'''+df['"+df.columns+"'][k]+'''"
display(var)

,headers,postman_var,py_var
0,campaignid,{{campaignid}},'''+df['campaignid'][k]+'''
1,uid,{{uid}},'''+df['uid'][k]+'''
2,first name,{{first name}},'''+df['first name'][k]+'''
3,last name,{{last name}},'''+df['last name'][k]+'''
4,email,{{email}},'''+df['email'][k]+'''
5,mobile phone - b,{{mobile phone - b}},'''+df['mobile phone - b'][k]+'''
6,home phone - b,{{home phone - b}},'''+df['home phone - b'][k]+'''
7,first name (co-borrower),{{first name (co-borrower)}},'''+df['first name (co-borrower)'][k]+'''
8,last name (co-borrower),{{last name (co-borrower)}},'''+df['last name (co-borrower)'][k]+'''
9,email (co-borrower),{{email (co-borrower)}},'''+df['email (co-borrower)'][k]+'''


## 4. Content-type

Run the cell below to display a widget. You will be prompted to choose the content type in which the post bodies will be processed (currently accepting json/xml).

In [35]:
content = widgets.Dropdown(
        options=['application/json', 'application/xml'],
        value='application/json',
        description='Content-type:',
        disabled=False,
    )
display(content)

Dropdown(description='Content-type:', options=('application/json', 'application/xml'), value='application/json…

## 5. HTTP POST

- Map the notebook's variables above to use in your HTTP POST below
- Only change the URL and PAYLOAD in this cell
- (Opt.) Try verifying the request contents using `print(url)` and `print(payload)` to make sure the values are being mapped correctly.
- When confident, run the cell below to make the requests

In [ ]:
for k in range(len(df)):
    
    ## BEGIN REQUEST ##
    
    URL = '''https://app.insellerate.com/api/integration/CampaignPost/1011/10047'''
    
    PAYLOAD = '''    
    <MortgageLead>
        <LeadSourceId>'''+df['ref id'][k]+'''</LeadSourceId>
        <ContactInfo>
            <Email></Email>
            <Phone1></Phone1>
            <Phone1Type>Home</Phone1Type>
        </ContactInfo>
        <Borrower>
            <FirstName>'''+df['first name'][k]+'''</FirstName>
            <LastName>'''+df['last name'][k]+'''</LastName>
            <MailCity></MailCity>
            <MailState></MailState>
            <MailZip></MailZip>
            <CreditRating></CreditRating>
            <age></age>
            <UniversalLeadId></UniversalLeadId>
        </Borrower>
        <Property>
            <PropertyValue></PropertyValue>
            <PurchaseSituation></PurchaseSituation>
            <PropertyType></PropertyType>
            <PropertyUsage></PropertyUsage>
        </Property>
        <Loan>
            <LoanType></LoanType>
            <LoanAmount></LoanAmount>
            <LoanToValue></LoanToValue>
            <DownPayment></DownPayment>
            <loanownedby></loanownedby>
            <FirstMortgageBalance></FirstMortgageBalance>
            <FirstMortgageRate></FirstMortgageRate>
            <DesiredRateType></DesiredRateType>
        </Loan>
        <grossannualincome></grossannualincome>
        <ReasonForReverseMortgage></ReasonForReverseMortgage>
        <workingwithrealestateagent></workingwithrealestateagent>
        <LatePayments></LatePayments>
        <FhaLoan></FhaLoan>
        <MonthlyIncome></MonthlyIncome>
        <MonthlyExpenses></MonthlyExpenses>
        <ProofOfIncome></ProofOfIncome>
        <RecentBankruptcy></RecentBankruptcy>
        <EmploymentStatus></EmploymentStatus>
        <AdditionalCash></AdditionalCash>
        <SecondMortgage></SecondMortgage>
        <ExistingRateType></ExistingRateType>
        <FirstPurchase></FirstPurchase>
        <YearOfPurchase></YearOfPurchase>
        <DeviceType></DeviceType>
        <StickyBarLead></StickyBarLead>
        <StickyBarURL></StickyBarURL>
        <PubID></PubID>
    </MortgageLead>
    '''
    
    ## END REQUEST ##
    
    r = requests.post(URL, data=PAYLOAD, headers={'Content-type': content.value, 'Accept': 'text/plain'})
    post_data = {'RequestUrl' : URL, 'RequestBody' : PAYLOAD.replace('\n', ''), 'ResponseCode' : r,'ResponseBody' : r.text}
    post_data.update(df.to_dict('records')[k])
    logs = logs.append(post_data, 
                       ignore_index = True)

    print('Running '+str(k+1)+ '/'+str(len(df))+'...')
    
display(logs)

## 6. Export Request Log

Ah, yes. Something so simple yet important to have once a bulk request is completed, but is not provided by Postman. 

Steps 1-5 were essentially replicating the Postman runner, while step 6 provides a much needed log that can allow the user to export and juxtapose the CSV data that was provided, the request that was sent, and the response that was received. With this information, the user could parse through the response and even use the given identifers to fetch the posted data in the endpoint's server (different notebook I'm working on that works adjacently with this task)-- providing a clear perception on the flow of data from beginning to end.

Run the cell below to export the logs to your Downloads folder.

In [31]:
# logs.to_csv('Downloads/'+filename[:-4]+'-logs.csv',header=True)

# SPECIFY FILE PATH WHEN IPYWIDGETS ARE DOWN
logs.to_csv('Downloads/20211104-logs.csv',header=True)